<a href="https://colab.research.google.com/github/bthodla/pandas_vs_terality/blob/main/notebooks/pandas_vs_terality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
import terality as te
from codetiming import Timer
import random

In [2]:
file_path = "C:\\\\Users\\\\bhask\\\\Google Drive\\\\datasets\\\\Chicago_taxi_trips\\\\"
timer = Timer(name="Terality Tests")

In [3]:
# Disabling Terality cache for testing
cache_disabler = te.disable_cache()
cache_disabler.__enter__()

In [72]:
def get_slice_range(limit):
    start = random.randint(1, limit)
    end = random.randint(start, start + 1000)
    return start, end

In [4]:
# A list of dataframe columns renamed by converting to lowercase and replacing spaces with '_'
columns = [
'trip_id',
'taxi_id',
'trip_start_timestamp',
'trip_end_timestamp',
'trip_seconds',
'trip_miles',
'pickup_census_tract',
'dropoff_census_tract',
'pickup_community_area',
'dropoff_community_area',
'fare',
'tips',
'tolls',
'extras',
'trip_total',
'payment_type',
'company',
'pickup_centroid_latitude',
'pickup_centroid_longitude',
'pickup_centroid_location',
'dropoff_centroid_latitude',
'dropoff_centroid_longitude',
'dropoff_centroid__location'
]

In [5]:
# Loading data into dataframes using Pandas
timer.start()
taxi_trips_pdf = pd.read_csv(file_path + "Taxi_Trips_2019_2020.csv")
timer.stop()

Elapsed time: 147.5039 seconds


147.503921

In [6]:
# Loading data into dataframes using Terality
timer.start()
taxi_trips_tdf = te.read_csv(file_path + "Taxi_Trips_2019_2020.csv")
timer.stop()

C:\\Users\\bhask\\Google Drive\\datasets\\Chicago_taxi_trips\\Taxi_Trips_2019_2020.csv:   0%| | 0.00/8.48G [00:02<?, ?B
INFO:terality:The result of te.read_csv was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Elapsed time: 47.5288 seconds


47.52880470000002

In [7]:
timer.start()
taxi_trips_pdf.head()
timer.stop()

Elapsed time: 0.0015 seconds


0.0014795999999819287

In [8]:
timer.start()
taxi_trips_tdf.head()
timer.stop()

INFO:terality:The result of te.dataframe.head was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Elapsed time: 1.7167 seconds


1.7166523999999868

In [9]:
# Rename the columns of the Pandas dataframe
timer.start()
taxi_trips_pdf.columns = columns
timer.stop()

Elapsed time: 0.0003 seconds


0.0003424999999879219

In [10]:
# Rename the columns of the Terality dataframe
timer.start()
taxi_trips_tdf.columns = columns
timer.stop()

INFO:terality:The result of te.dataframe.columns was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Elapsed time: 1.4328 seconds


1.4327554999999847

In [11]:
# Row count before cleanup - using Pandas
timer.start()
print("Pandas DF Row Count before: % d" % taxi_trips_pdf.shape[0])
timer.stop()

Pandas DF Row Count before:  20366397
Elapsed time: 0.0008 seconds


0.0008144999999899483

In [12]:
# Remove all the rows with NaN values using Pandas. There is plenty of data even otherwise
timer.start()
taxi_trips_pdf.dropna(how='any',axis=0,inplace=True)
timer.stop()

Elapsed time: 13.6855 seconds


13.685485999999969

In [66]:
# Row count after cleanup - using Pandas
timer.start()
row_count = taxi_trips_pdf.shape[0]
print("Pandas DF Row Count after: % d" % row_count)
timer.stop()

Pandas DF Row Count after:  12199631
Elapsed time: 0.0017 seconds


0.001719500000035623

In [14]:
# Row count before cleanup - using Terality
timer.start()
print("Terality DF Row Count after: % d" % taxi_trips_tdf.shape[0])
timer.stop()

INFO:terality:The result of te.dataframe.shape was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Terality DF Row Count after:  20366397
Elapsed time: 1.6042 seconds


1.6041942999999605

In [15]:
# Remove all the rows with NaN values using Terality. There is plenty of data even otherwise
timer.start()
taxi_trips_tdf.dropna(how='any',axis=0,inplace=True)
timer.stop()

INFO:terality:The result of te.dataframe.dropna was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Elapsed time: 1.1060 seconds


1.1059541000000195

In [16]:
# Row count after cleanup - using Terality
timer.start()
print("Terality DF Row Count after: % d" % taxi_trips_tdf.shape[0])
timer.stop()

INFO:terality:The result of te.dataframe.shape was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


Terality DF Row Count after:  12199631
Elapsed time: 0.8688 seconds


0.8688448000000335

In [37]:
# Adding a new column to store datetime version of string datetimes - Pandas
timer.start()
taxi_trips_pdf['trip_start_dt'] = pd.to_datetime(taxi_trips_pdf['trip_start_timestamp'], infer_datetime_format=True)
taxi_trips_pdf['trip_end_dt'] = pd.to_datetime(taxi_trips_pdf['trip_end_timestamp'], infer_datetime_format=True)
taxi_trips_pdf['trip_start_ym'] = taxi_trips_pdf['trip_start_dt'].dt.to_period('M')
taxi_trips_pdf['trip_end_ym'] = taxi_trips_pdf['trip_end_dt'].dt.to_period('M')
timer.stop()

Elapsed time: 21.8859 seconds


21.88585409999996

In [38]:
taxi_trips_pdf.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
16,e066e483f0fc2a583895d24c9dec9864a6608052,4794c36af4c1852d90b5c8cf5f9f6cd38a8b9c1b68cf48...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,600.0,0.0,1.703108e+10,1.703184e+10,8.0,32.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
19,5431595d222a3e00a9e4b3e93c928c306dc71be3,86b0677bb9bcda0454863c70a616d1adc45129237848d1...,01/01/2019 12:00:00 AM,01/01/2019 12:00:00 AM,120.0,0.3,1.703184e+10,1.703132e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-01-01,2019-01-01 00:00:00,2019-01,2019-01
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
23,da18ec450c1fed02fbbc675faa621f8ac50ab5dd,0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,420.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01


In [33]:
# Adding a new column to store datetime version of string datetimes - Terality
timer.start()
taxi_trips_tdf['trip_start_dt'] = te.to_datetime(taxi_trips_tdf['trip_start_timestamp'], infer_datetime_format=True)
taxi_trips_tdf['trip_end_dt'] = te.to_datetime(taxi_trips_tdf['trip_end_timestamp'], infer_datetime_format=True)
taxi_trips_tdf['trip_start_ym'] = taxi_trips_tdf['trip_start_dt'].dt.to_period('M')
taxi_trips_tdf['trip_end_ym'] = taxi_trips_tdf['trip_end_dt'].dt.to_period('M')
timer.stop()

Elapsed time: 20.7048 seconds


20.704838499999823

In [34]:
taxi_trips_tdf.head()
# timer.stop()


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
16,e066e483f0fc2a583895d24c9dec9864a6608052,4794c36af4c1852d90b5c8cf5f9f6cd38a8b9c1b68cf48...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,600.0,0.0,1.703108e+10,1.703184e+10,8.0,32.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
19,5431595d222a3e00a9e4b3e93c928c306dc71be3,86b0677bb9bcda0454863c70a616d1adc45129237848d1...,01/01/2019 12:00:00 AM,01/01/2019 12:00:00 AM,120.0,0.3,1.703184e+10,1.703132e+10,32.0,32.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-01-01,2019-01-01 00:00:00,2019-01,2019-01
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01
23,da18ec450c1fed02fbbc675faa621f8ac50ab5dd,0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,420.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01,2019-01-01 00:15:00,2019-01,2019-01


In [17]:
# Describe - Pandas
timer.start()
taxi_trips_pdf.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
count,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07
mean,7.806322e+02,2.882408e+00,1.703141e+10,1.703138e+10,2.628359e+01,2.319189e+01,1.279652e+01,1.781760e+00,7.331033e-04,8.851525e-01,1.560699e+01,4.189265e+01,-8.765359e+01,4.189144e+01,-8.764459e+01
std,1.207102e+03,5.479885e+00,3.436317e+05,3.341613e+05,1.881160e+01,1.634532e+01,6.189516e+01,2.826719e+00,3.181317e-01,1.609252e+01,6.459551e+01,3.113462e-02,7.520429e-02,2.604625e-02,5.721153e-02
min,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.165022e+01,-8.790304e+01,4.166376e+01,-8.790304e+01
25%,3.400000e+02,5.400000e-01,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,4.188099e+01,-8.763784e+01,4.188099e+01,-8.764090e+01
50%,5.140000e+02,1.080000e+00,1.703132e+10,1.703128e+10,2.800000e+01,2.800000e+01,7.250000e+00,9.000000e-01,0.000000e+00,0.000000e+00,9.000000e+00,4.188530e+01,-8.763186e+01,4.188530e+01,-8.763186e+01
75%,8.230000e+02,1.900000e+00,1.703184e+10,1.703184e+10,3.200000e+01,3.200000e+01,1.025000e+01,2.000000e+00,0.000000e+00,1.000000e+00,1.250000e+01,4.189322e+01,-8.762197e+01,4.189503e+01,-8.762197e+01
max,8.640000e+04,9.959000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.800260e+03,4.000000e+02,9.606800e+02,9.327760e+03,9.800300e+03,4.202122e+01,-8.753071e+01,4.202122e+01,-8.753071e+01


In [18]:
timer.stop()

Elapsed time: 8.1017 seconds


8.101670899999988

In [19]:
# Describe - Terality
timer.start()
taxi_trips_tdf.describe()

INFO:terality:The result of te.dataframe.describe was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].
INFO:terality:The result of te.dataframe._repr_html_ was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
count,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07,1.219963e+07
mean,7.806322e+02,2.882408e+00,1.703141e+10,1.703138e+10,2.628359e+01,2.319189e+01,1.279652e+01,1.781760e+00,7.331033e-04,8.851525e-01,1.560699e+01,4.189265e+01,-8.765359e+01,4.189144e+01,-8.764459e+01
std,1.207102e+03,5.479885e+00,3.436317e+05,3.341613e+05,1.881160e+01,1.634532e+01,6.189516e+01,2.826719e+00,3.181317e-01,1.609252e+01,6.459551e+01,3.113462e-02,7.520429e-02,2.604625e-02,5.721153e-02
min,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.165022e+01,-8.790304e+01,4.166376e+01,-8.790304e+01
25%,3.400000e+02,5.400000e-01,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,4.188099e+01,-8.763784e+01,4.188099e+01,-8.764090e+01
50%,5.140000e+02,1.080000e+00,1.703132e+10,1.703128e+10,2.800000e+01,2.800000e+01,7.250000e+00,9.000000e-01,0.000000e+00,0.000000e+00,9.000000e+00,4.188530e+01,-8.763186e+01,4.188530e+01,-8.763186e+01
75%,8.230000e+02,1.900000e+00,1.703184e+10,1.703184e+10,3.200000e+01,3.200000e+01,1.025000e+01,2.000000e+00,0.000000e+00,1.000000e+00,1.250000e+01,4.189322e+01,-8.762197e+01,4.189503e+01,-8.762197e+01
max,8.640000e+04,9.959000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.800260e+03,4.000000e+02,9.606800e+02,9.327760e+03,9.800300e+03,4.202122e+01,-8.753071e+01,4.202122e+01,-8.753071e+01


In [20]:
timer.stop()

Elapsed time: 3.1106 seconds


3.110634800000014

In [21]:
# Query 1: Number of trips grouped by taxi - Using Pandas
timer.start()
taxi_trips_pdf.groupby('taxi_id').count()['trip_id']


taxi_id
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b    3372
001330b81e23412049f9c3eff5b6e972a91afe59c9aa36ef29747881b4bf803adfd02b1a71cca5cb8c2214275330aa4520df34d7848100414acb4b4cad642e62    3072
0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be70eff37b63ef9ca4102c972145ddd8537811752d51b222b4618dfbae451b966d7a3ad9afa4ec878a6e    4093
0041f8f0c91881c1e1913f2548522495fe3c4c719aa67fd2483ebbcc22e0aa1bfff2cddb32c35cde3ad9766b0e8d44c04ff5dfdcf1035462ba81a13e3ba6b8f9    3431
0044e6c0d091476299b99345501f756b23632a96cbaf40e872fbf14f976410d3f938aacc643ed608b2aa42809222d4458e1aab5e0848e9b952f35616785c3a36    2104
                                                                                                                                    ... 
ffd70d111c7da2e5eff27ad900c8eb94d74fe286bb7aa58029ee2baac3f3b310dea5da2a6a77a366b7b6d4b104b91ef75e7959ac6b1f558be7844417cbe3e12f     603
ffda53354c610fd3af1aee46d723028a4

In [22]:
timer.stop()

Elapsed time: 14.2906 seconds


14.290632199999948

In [23]:
# Query 1: Number of trips grouped by taxi - Using Terality
timer.start()
taxi_trips_pdf.groupby('taxi_id').count()['trip_id']



taxi_id
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b    3372
001330b81e23412049f9c3eff5b6e972a91afe59c9aa36ef29747881b4bf803adfd02b1a71cca5cb8c2214275330aa4520df34d7848100414acb4b4cad642e62    3072
0013da5489fe976daf4f4a7d246073ecf2caed9b12b3be70eff37b63ef9ca4102c972145ddd8537811752d51b222b4618dfbae451b966d7a3ad9afa4ec878a6e    4093
0041f8f0c91881c1e1913f2548522495fe3c4c719aa67fd2483ebbcc22e0aa1bfff2cddb32c35cde3ad9766b0e8d44c04ff5dfdcf1035462ba81a13e3ba6b8f9    3431
0044e6c0d091476299b99345501f756b23632a96cbaf40e872fbf14f976410d3f938aacc643ed608b2aa42809222d4458e1aab5e0848e9b952f35616785c3a36    2104
                                                                                                                                    ... 
ffd70d111c7da2e5eff27ad900c8eb94d74fe286bb7aa58029ee2baac3f3b310dea5da2a6a77a366b7b6d4b104b91ef75e7959ac6b1f558be7844417cbe3e12f     603
ffda53354c610fd3af1aee46d723028a4

In [24]:
timer.stop()

Elapsed time: 12.3695 seconds


12.369518699999958

In [40]:
# Query 2: Number of trips grouped by taxi and year/month - Using Pandas
timer.start()
taxi_trips_pdf.groupby(['taxi_id', 'trip_start_ym']).count()['trip_id']

taxi_id                                                                                                                           trip_start_ym
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b  2019-01          230
                                                                                                                                  2019-02          248
                                                                                                                                  2019-03          212
                                                                                                                                  2019-04          202
                                                                                                                                  2019-05          270
                                                                                                     

In [47]:
timer.stop()

Elapsed time: 25.7059 seconds


25.7058820000002

In [43]:
# Query 2: Number of trips grouped by taxi and year/month - Using Terality
timer.start()
taxi_trips_tdf.groupby(['taxi_id', 'trip_start_ym']).count()['trip_id']

taxi_id                                                                                                                           trip_start_ym
0008de7a146802839c9e6059f482d292ebdae13c5c31dd6e5983a80882e2a5dbcd6ea098c2fcd56f34ce02645eb94c6b39512e9304837746d4e289b6236c2c5b  2019-01          230
                                                                                                                                  2019-02          248
                                                                                                                                  2019-03          212
                                                                                                                                  2019-04          202
                                                                                                                                  2019-05          270
                                                                                                     

In [59]:
timer.stop()

Elapsed time: 9.9748 seconds


9.974799299999177

In [61]:
# Query 3: Number of trips by Taxi Company - using Pandas
timer.start()
taxi_trips_pdf.groupby('company').count()['trip_id'].sort_values(ascending=False).head(10)


company
Taxi Affiliation Services            2769009
Chicago Carriage Cab Corp            1506931
Flash Cab                            1220891
Sun Taxi                             1046724
Medallion Leasin                      932280
City Service                          906870
Star North Management LLC             742670
Blue Ribbon Taxi Association Inc.     581908
Choice Taxi Association               400096
Taxicab Insurance Agency, LLC         340868
Name: trip_id, dtype: int64

In [62]:
timer.stop()

Elapsed time: 7.1848 seconds


7.184787299999698

In [ ]:
# Query 3: Number of trips by Taxi Company - using Terality
timer.start()
taxi_trips_tdf.groupby('company').count()['trip_id'].sort_values(ascending=False).head(10)


In [ ]:
timer.stop()

In [ ]:
# Query 4: Sort by Taxi Company - using Pandas
timer.start()
taxi_trips_pdf.sort_values(by="company")
timer.stop()

In [ ]:
# Query 4: Sort by Taxi Company - using Terality
timer.start()
taxi_trips_tdf.sort_values(by="company")
timer.stop()

In [83]:
range = get_slice_range(row_count)
print("Start: %d, End: %d" % range)


Start: 8802434, End: 8802590


In [84]:
# Query 5: Slicing using random numbers for start and end - using Pandas
timer.start()
taxi_trips_pdf[range[0]:range[1]]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905214,a5965ce576d4d0f9043b9d21bbe1226ad30e279e,71ebe150c7d317935a0843c988eea605df50b3ba7cb6bf...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,1038.0,1.20,1.703108e+10,1.703108e+10,8.0,8.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905216,9a6dededb445105aa8a39a26a78304f66bef686f,d12b9e3cb8201f4c58e7d3ad3a970bb83f46788415f5d9...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,480.0,1.30,1.703128e+10,1.703132e+10,28.0,32.0,...,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905217,9b585ea357d98713942a64e633fa8415097aaa04,6143d93de4a09820f2d0d6b37acc8161d90a411b9bc1bd...,10/30/2019 05:30:00 PM,10/30/2019 05:30:00 PM,0.0,0.00,1.703183e+10,1.703183e+10,21.0,21.0,...,41.942859,-87.717504,POINT (-87.717503858 41.942859303),41.942859,-87.717504,POINT (-87.717503858 41.942859303),2019-10-30 17:30:00,2019-10-30 17:30:00,2019-10,2019-10
13905218,9b699a15ac84cca23a6bb39bc14b275107886f59,d84eb9fc599449340e9a37c4aeac90c89b3ec1fb0ae210...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,624.0,1.98,1.703108e+10,1.703107e+10,8.0,7.0,...,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.922083,-87.634156,POINT (-87.6341560931 41.922082541),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905219,9bd8f3907acdcafe29527d9938899114b4fb6065,693df44ce3a8256a0fcdace4d211a99ab8af09c75f35df...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,812.0,1.56,1.703108e+10,1.703132e+10,8.0,32.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905402,d28eb25c9a5fa4744787c95929d3b279329cce3e,1ed802e8138d030626cb7276ceadd721c743526af747ac...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,979.0,1.82,1.703108e+10,1.703184e+10,8.0,32.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905404,d3594089126fce46d5226cfba3b883c6a125dd31,6834529dad6f57a5e4e090ebae5a902a2320ec7448e2b0...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,600.0,1.10,1.703132e+10,1.703184e+10,32.0,32.0,...,41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905405,d37ee46a9969d42cad12f151f3336f03a20efe36,8c91a71ec9d69d423b9e9d76356664ea513c485bc773a4...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,840.0,0.00,1.703132e+10,1.703132e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905406,d3a551f609a6754209ff017eeaeddf128886d216,8e84db8f70dbd552279117817212bfd78369e3b9951883...,10/30/2019 05:30:00 PM,10/30/2019 05:30:00 PM,456.0,0.91,1.703184e+10,1.703108e+10,32.0,8.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),2019-10-30 17:30:00,2019-10-30 17:30:00,2019-10,2019-10


In [85]:
timer.stop()

Elapsed time: 2.3647 seconds


2.364680999999109

In [86]:
# Query 5: Slicing using random numbers for start and end - using Terality
timer.start()
taxi_trips_tdf[range[0]:range[1]]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905214,a5965ce576d4d0f9043b9d21bbe1226ad30e279e,71ebe150c7d317935a0843c988eea605df50b3ba7cb6bf...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,1038.0,1.20,1.703108e+10,1.703108e+10,8.0,8.0,...,41.891972,-87.612945,POINT (-87.6129454143 41.8919715078),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905216,9a6dededb445105aa8a39a26a78304f66bef686f,d12b9e3cb8201f4c58e7d3ad3a970bb83f46788415f5d9...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,480.0,1.30,1.703128e+10,1.703132e+10,28.0,32.0,...,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905217,9b585ea357d98713942a64e633fa8415097aaa04,6143d93de4a09820f2d0d6b37acc8161d90a411b9bc1bd...,10/30/2019 05:30:00 PM,10/30/2019 05:30:00 PM,0.0,0.00,1.703183e+10,1.703183e+10,21.0,21.0,...,41.942859,-87.717504,POINT (-87.717503858 41.942859303),41.942859,-87.717504,POINT (-87.717503858 41.942859303),2019-10-30 17:30:00,2019-10-30 17:30:00,2019-10,2019-10
13905218,9b699a15ac84cca23a6bb39bc14b275107886f59,d84eb9fc599449340e9a37c4aeac90c89b3ec1fb0ae210...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,624.0,1.98,1.703108e+10,1.703107e+10,8.0,7.0,...,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.922083,-87.634156,POINT (-87.6341560931 41.922082541),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905219,9bd8f3907acdcafe29527d9938899114b4fb6065,693df44ce3a8256a0fcdace4d211a99ab8af09c75f35df...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,812.0,1.56,1.703108e+10,1.703132e+10,8.0,32.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905402,d28eb25c9a5fa4744787c95929d3b279329cce3e,1ed802e8138d030626cb7276ceadd721c743526af747ac...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,979.0,1.82,1.703108e+10,1.703184e+10,8.0,32.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905404,d3594089126fce46d5226cfba3b883c6a125dd31,6834529dad6f57a5e4e090ebae5a902a2320ec7448e2b0...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,600.0,1.10,1.703132e+10,1.703184e+10,32.0,32.0,...,41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905405,d37ee46a9969d42cad12f151f3336f03a20efe36,8c91a71ec9d69d423b9e9d76356664ea513c485bc773a4...,10/30/2019 05:30:00 PM,10/30/2019 05:45:00 PM,840.0,0.00,1.703132e+10,1.703132e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),2019-10-30 17:30:00,2019-10-30 17:45:00,2019-10,2019-10
13905406,d3a551f609a6754209ff017eeaeddf128886d216,8e84db8f70dbd552279117817212bfd78369e3b9951883...,10/30/2019 05:30:00 PM,10/30/2019 05:30:00 PM,456.0,0.91,1.703184e+10,1.703108e+10,32.0,8.0,...,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),2019-10-30 17:30:00,2019-10-30 17:30:00,2019-10,2019-10


In [87]:
timer.stop()

Elapsed time: 10.6349 seconds


10.634887200001685

In [88]:
# Query 6: Selecting a single row from an unindexed dataframe - using Pandas
timer.start()
taxi_trips_pdf[taxi_trips_pdf['trip_id'] == 'd3e437e3af9c691e6a9fe1f9802605d989605fdb']

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905408,d3e437e3af9c691e6a9fe1f9802605d989605fdb,ac66734513cb7344a5501fa22f825c940448a4041e0302...,10/30/2019 05:30:00 PM,10/30/2019 06:00:00 PM,1396.0,1.14,1.703108e+10,1.703108e+10,8.0,8.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 18:00:00,2019-10,2019-10


In [89]:
timer.stop()

Elapsed time: 7.6062 seconds


7.606247600000643

In [90]:
# Query 6: Selecting a single row from an unindexed dataframe - using Terality
timer.start()
taxi_trips_tdf[taxi_trips_tdf['trip_id'] == 'd3e437e3af9c691e6a9fe1f9802605d989605fdb']

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
13905408,d3e437e3af9c691e6a9fe1f9802605d989605fdb,ac66734513cb7344a5501fa22f825c940448a4041e0302...,10/30/2019 05:30:00 PM,10/30/2019 06:00:00 PM,1396.0,1.14,1.703108e+10,1.703108e+10,8.0,8.0,...,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-10-30 17:30:00,2019-10-30 18:00:00,2019-10,2019-10


In [91]:
timer.stop()

Elapsed time: 406.1916 seconds


406.19155070000124

In [95]:
# Query 7: Range queries using isin() - Using Pandas
timer.start()
taxi_trips_pdf[taxi_trips_pdf['company'].isin(['Taxi Affiliation Services', 'Blue Diamond', 'Chicago Taxicab', 'Nova Taxi Affiliation Llc', 'U Taxicab'])]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location,trip_start_dt,trip_end_dt,trip_start_ym,trip_end_ym
20,509e870adce2a514a4287e1a305fe4ddabc59836,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
22,31004535c12d4372c165e34386cde0521a0a72e8,ee8a2f86372e99ecf87d44e13a20ebda7db8ae69d5b444...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,1.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
30,601ea33fd396dadf3c05667ce8badbab55968930,72cd7073f26565b4a99646e795893b87b06e8086ad9a49...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,180.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
35,d1c76d15cec1a2ee58b408aff06e303aa86dd37f,d38890256d8a1e8146f0b15dc23e3b2a140b4fa3834698...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,300.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.898332,-87.620763,POINT (-87.6207628651 41.8983317935),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
36,e933481970a4efe76e013706d4056104dbf3a599,3506f62ee1c70c02fe00b742679684b319d5b712182f82...,01/01/2019 12:00:00 AM,01/01/2019 12:15:00 AM,360.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),2019-01-01 00:00:00,2019-01-01 00:15:00,2019-01,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20365970,8b2fb9ed7cb74a7645c95538373111ac90be222a,7476a493f34aa1075570e89104cdaf3aaf5c0fd4679382...,12/31/2020 08:45:00 PM,12/31/2020 09:00:00 PM,1200.0,17.1,1.703198e+10,1.703128e+10,76.0,28.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.879255,-87.642649,POINT (-87.642648998 41.8792550844),2020-12-31 20:45:00,2020-12-31 21:00:00,2020-12,2020-12
20366091,91fddaa11017287697ed65e5b6fb6399ea4f7ac4,5f88daef22c65f61d051338809a96d6bc7f471c4e2cd73...,12/31/2020 09:30:00 PM,12/31/2020 09:30:00 PM,120.0,0.2,1.703132e+10,1.703132e+10,32.0,32.0,...,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),2020-12-31 21:30:00,2020-12-31 21:30:00,2020-12,2020-12
20366290,debd80beca876e44a7d485a2fd2fd66816f8fc2b,42574f4a000f78c9ccbf2cfc282f31617096a9fea4a0d5...,12/31/2020 10:45:00 PM,12/31/2020 11:00:00 PM,180.0,0.0,1.703198e+10,1.703198e+10,76.0,76.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),2020-12-31 22:45:00,2020-12-31 23:00:00,2020-12,2020-12
20366292,e9fc3c32d17dd7dcd54cabaca1781d4684b8832d,be2af1b95b937dfe54bf9a4f4331745a673529a7edfab2...,12/31/2020 10:45:00 PM,12/31/2020 11:00:00 PM,480.0,2.8,1.703198e+10,1.703198e+10,76.0,76.0,...,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),2020-12-31 22:45:00,2020-12-31 23:00:00,2020-12,2020-12


In [97]:
timer.stop()

TimerError: ignored

In [ ]:
# Query 7: Range queries using isin() - Using Terality
timer.start()
taxi_trips_pdf['company'].isin(['Taxi Affiliation Services', 'Blue Diamond', 'Chicago Taxicab', 'Nova Taxi Affiliation Llc', 'U Taxicab'])

In [92]:
timer.stop()

array(['Blue Ribbon Taxi Association Inc.', 'Star North Management LLC',
       'Taxi Affiliation Services', 'Taxicab Insurance Agency, LLC',
       'Choice Taxi Association', 'Top Cab Affiliation',
       'Chicago Independents', 'KOAM Taxi Association',
       '1085 - 72312 N and W Cab Co', 'Chicago Medallion Management',
       'Chicago Carriage Cab Corp', 'Flash Cab', 'Sun Taxi', 'Globe Taxi',
       '24 Seven Taxi', 'City Service', 'Taxi Affiliation Service Yellow',
       'Medallion Leasin', 'Blue Diamond', 'Chicago Taxicab',
       'Patriot Taxi Dba Peace Taxi Associat', 'Gold Coast Taxi',
       'Checker Taxi Affiliation', 'Nova Taxi Affiliation Llc',
       '5 Star Taxi', 'Metro Jet Taxi A', '6742 - 83735 Tasha ride inc',
       'Setare Inc', '1469 - 64126 Omar Jada', '6743 - 78771 Luhak Corp',
       'American United Taxi Affiliation', 'Leonard Cab Co',
       '4053 - 40193 Adwar H. Nikola', '312 Medallion Management Corp',
       '3011 - 66308 JBL Cab Inc.', '4623 - 27290 Jay